In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.plots as plots
import numpy as np

In [2]:
import lsst.sims.maf
lsst.sims.maf.__version__

'2.8.0.sims-22-gab2f692'

In [3]:
runName = 'mothra_2048'
opsdb = db.OpsimDatabase('../../../mothra_2048/data/mothra_2048.db')
outDir = 'stackers_test'
sqlconstraint = 'night<730'
resultsDb = db.ResultsDb(outDir=outDir)

In [4]:
slicer = slicers.HealpixSlicer(nside=64)
stackerList = [stackers.NormAirmassStacker(degrees=True),stackers.HourAngleStacker()]

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [5]:
metric = metrics.MedianMetric('normairmass')

In [6]:
bundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint, 
                                    runName=runName,
                                    stackerList=stackerList)
bgroup = metricBundles.MetricBundleGroup({'normairmass': bundle}, 
                                         opsdb, outDir=outDir, 
                                         resultsDb=resultsDb)

In [7]:
bgroup.runAll()

Querying database SummaryAllProps with constraint night<730 for columns ['observationStartLST', 'fieldDec', 'fieldRA', 'airmass']
Found 327472 visits
Running:  ['normairmass']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [8]:
print(bgroup.simData.dtype.names)

('observationStartLST', 'fieldDec', 'fieldRA', 'airmass', 'normairmass', 'HA')


In [9]:
bgroup.simData

array([ ( 305.02475404, -24.889283,  305.088793,  1.00523775,  1.00084987, -0.00426926),
       ( 305.46762996,  35.966846,  304.770694,  2.45369622,  0.98972754,  0.0464624 ),
       ( 306.24892991, -62.802603,  305.4822  ,  1.17545707,  0.99073099,  0.05111533),
       ...,
       (  78.14658242, -34.120161,  133.791524,  1.48427365,  1.48087948, -3.70966277),
       (  78.32206155, -22.701863,  131.690357,  1.49896946,  1.48599908, -3.55788636),
       (  78.48918454, -14.112463,  127.206247,  1.49567871,  1.43678378, -3.24780416)],
      dtype=[('observationStartLST', '<f8'), ('fieldDec', '<f8'), ('fieldRA', '<f8'), ('airmass', '<f8'), ('normairmass', '<f8'), ('HA', '<f8')])

In [10]:
telescope_lat = -30.2444

Use this field (These values are just taken from)

RA = 306.24892991

DEC = -62.802603

HA = 0.05111533

Opsim Airmass = 1.17545707

In [11]:
# convert HA in hours to radians
ha_rads = (np.pi*0.05111533)/12.0

In [12]:
dec = np.radians(-62.802603)
min_z_possible = np.abs(dec - np.radians(telescope_lat))
min_airmass_possible = 1./np.cos(min_z_possible)

In [13]:
min_airmass_possible

1.1864574844694964

In [14]:
cos_z = np.sin(np.radians(telescope_lat))*np.sin(dec)+np.cos(np.radians(telescope_lat))*np.cos(dec)*np.cos(ha_rads)
calculated_airmass = 1./cos_z
calculated_airmass

1.1865072527175096

In [15]:
calculated_airmass/min_airmass_possible

1.0000419469291268

In [16]:
#opsim airmass value - calculated airmass value:
1.17545707 - calculated_airmass

-0.011050182717509571

In [17]:
#percent change
((1.17545707 - calculated_airmass)/1.17545707)*100

-0.94007539701212328

In [18]:
# opsdb.fetchMetricData(['airmass','fieldRA','fieldDec','observationStartMJD'])